# Center Cut Prediction Image

In [46]:
!pip install scikit-image

In [47]:
from itertools import groupby, chain
from PIL import Image
import matplotlib.pyplot as plt
import cv2 
import numpy as np
from skimage import io, color
import sys
from skimage import img_as_ubyte
import skimage.io as io
import os
np.set_printoptions(threshold=sys.maxsize)

### Commong Utility Functions

In [48]:
#from utilities.model_errors import ValidationError
def find_centermost_mask(input_mask, min_pixels = 10000):
    """
    Take in a binary mask and return a mask of the same shape, with only the centermost contiguous portion.
    If a mask contains no contours (objects), then just return it (it's all zeros anyway).
    """

    # Get all the bits aligned in memory
    input_mask = np.ascontiguousarray(input_mask, dtype=np.uint8)
    contours, _ = cv2.findContours(input_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     if not contours or len(contours) == 0:
#         output_mask = np.zeros(input_mask.shape)
#         return output_mask

    moments = [cv2.moments(contour) for contour in contours]
    
#     big_contour_area_idx = 0
#     if len(contours) > 1:
#         contour_areas = [m['m00'] for m in moments]
#         big_contour_area_idx = contour_areas.index(max(contour_areas))
#------------------------------------------------------------------    
    # (x, y) tuples of centroid locations for contours (that have area > 0)
    
#     for m in moments:
#         print('m[00] :',m['m00'])
    
    min_pixels = 10000
#     centroids = []
#     for m in moments:
#         if m['m00'] >= min_pixels:
#             print('selected countour area : ', m['m00'])
#             centroids.append((m['m10'] / m['m00'], m['m01'] / m['m00']))
#         else:
#             print('rejected countour area : ', m['m00'])
#             centroids.append((0,0))
            
#     print('centroids', centroids)
    
    min_pixels = 10000
    centroids = [
        (m['m10'] / m['m00'], m['m01'] / m['m00'])
        if m['m00'] >= min_pixels else (0, 0)
        for m in moments
    ]

    if not centroids:
        raise ValidationError(f'No identified footprint contours contain more than {min_pixels} pixels.')

    # dimensions come 1-indexed, but array indicies are 0-indexed
    center = [(dim - 1) / 2 for dim in input_mask.shape]

    # Cacluate (Manhattan) distance from center of image
    distances_from_center = [abs(x - center[0]) + abs(y - center[1]) for x, y in centroids]
    centermost_contour = contours[distances_from_center.index(min(distances_from_center))]
    idx = distances_from_center.index(min(distances_from_center))
#     print('center most contour idx : ', moments[idx]['m00'])
#------------------------------------------------------------------

#     centermost_contour = contours[big_contour_area_idx]
    output_mask = np.zeros(input_mask.shape)
    cv2.fillPoly(output_mask, centermost_contour.transpose((1, 0, 2)), 1)

    return output_mask

### Center cut the Prediction Image and Center Cut the Original Image

In [54]:
pred_img_path = '/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/run-1/resnet-34-out'
pred_cc_save_path = '/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc'


# pred_img_path = '/home/ec2-user/SageMaker/notebooks/bfp-related-nb/test_rotate_pred'
# pred_cc_save_path = '/home/ec2-user/SageMaker/notebooks/bfp-related-nb/test_rotate_pred_cc'

# label_img_path = '/home/ec2-user/SageMaker/data/footprint/190-test-images-label'

In [55]:
lst_pred_imgs = [img for img in os.listdir(pred_img_path) if os.path.isfile(os.path.join(pred_img_path, img))]
lst_pred_imgs, len(lst_pred_imgs)

(['image_427_pred_binary.png',
  'image_489_pred_binary.png',
  'image_99_pred_binary.png',
  'image_40_pred_binary.png',
  'image_359_pred_binary.png',
  'image_229_pred_binary.png',
  'image_523_pred_binary.png',
  'image_583_pred_binary.png',
  'image_493_pred_binary.png',
  'image_131_pred_binary.png',
  'image_125_pred_binary.png',
  'image_475_pred_binary.png',
  'image_233_pred_binary.png',
  'image_238_pred_binary.png',
  'image_308_pred_binary.png',
  'image_446_pred_binary.png',
  'image_89_pred_binary.png',
  'image_577_pred_binary.png',
  'image_234_pred_binary.png',
  'image_55_pred_binary.png',
  'image_80_pred_binary.png',
  'image_350_pred_binary.png',
  'image_420_pred_binary.png',
  'image_26_pred_binary.png',
  'image_382_pred_binary.png',
  'image_393_pred_binary.png',
  'image_419_pred_binary.png',
  'image_599_pred_binary.png',
  'image_222_pred_binary.png',
  'image_119_pred_binary.png',
  'image_3_pred_binary.png',
  'image_197_pred_binary.png',
  'image_21_pred

In [ ]:
for idx, pred_img_name in enumerate(lst_pred_imgs, start=1):
    if pred_img_name.startswith('...'):
        print('test run found')
        continue
    
    print('processing : {}, Image Name : {}'.format(idx, pred_img_name))
    pred_img_path_with_filename = os.path.join(pred_img_path, pred_img_name)
    pred_gray = cv2.imread(pred_img_path_with_filename)
    pred_gray = cv2.cvtColor(pred_gray, cv2.COLOR_BGR2GRAY)
    print(np.unique(pred_gray))
    print(pred_gray.shape)
    
#     lbl_gray[lbl_gray == 30] = 0
    unique_pixel_values = np.unique(pred_gray)
    if(len(unique_pixel_values) > 1):  # to handle no building scenario returns one pixel value that is '0'
        _, thresh = cv2.threshold(pred_gray, unique_pixel_values[0] + 1, 255, 0)
        print(np.unique(thresh))
        pred_cc_img_mask=find_centermost_mask(thresh,2000)
    else:
        pred_cc_img_mask = np.zeros(pred_gray.shape)
    
    pred_cc_img_mask = pred_cc_img_mask.astype(np.byte)
    io.imsave(os.path.join(pred_cc_save_path, pred_img_name.replace('_pred', '_cc')), pred_cc_img_mask)
#     plt.imshow(pred_cc_img_mask)
#     plt.show()

/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_427_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_489_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_99_cc_binary.png is a low contrast image
Lossy conv

processing : 1, Image Name : image_427_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 34136.0
m[00] : 44475.0
m[00] : 13352.0
processing : 2, Image Name : image_489_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 62093.0
processing : 3, Image Name : image_99_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 3693.0
m[00] : 8384.0
m[00] : 71384.5
processing : 4, Image Name : image_40_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 65275.5
processing : 5, Image Name : image_359_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 0.5
m[00] : 821.5
m[00] : 0.0
m[00] : 3557.5
m[00] : 52507.5
processing : 6, Image Name : image_229_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 23371.0
m[00] : 439.5
m[00] : 22181.5
processing : 7, Image Name : image_523_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 3.0
m[00] : 26.0
m[00] : 22737.0
m[00] : 26620.5
m[00] : 13187.0
processing : 8, Image Name : image_583_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]


/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_493_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_131_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_125_cc_binary.png is a low contrast image
Lossy con

processing : 9, Image Name : image_493_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 74477.0
processing : 10, Image Name : image_131_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 0.0
m[00] : 0.0
m[00] : 1.0
m[00] : 58234.0
m[00] : 3.0
m[00] : 1.0
m[00] : 20199.0
m[00] : 329.5
processing : 11, Image Name : image_125_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 349.0
m[00] : 55633.5
m[00] : 17600.5
m[00] : 17481.5
processing : 12, Image Name : image_475_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 63186.0
processing : 13, Image Name : image_233_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 0.0
m[00] : 9262.0
m[00] : 1621.0
m[00] : 24163.5
m[00] : 11908.0
processing : 14, Image Name : image_238_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 0.0
m[00] : 0.0
m[00] : 43828.5
processing : 15, Image Name : image_308_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 1580.0
m[00] : 186.5
m[00] : 947.0
m[00] : 50391.5
m[00] : 310.5
proce

/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_446_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_89_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_577_cc_binary.png is a low contrast image
Lossy conv

...

/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_290_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_214_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_24_cc_binary.png is a low contrast image
Lossy conv

processing : 381, Image Name : image_290_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 20627.5
m[00] : 110886.0
processing : 382, Image Name : image_214_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 176541.0
processing : 383, Image Name : image_24_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 13902.0
m[00] : 65303.5
processing : 384, Image Name : image_241_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 0.0
m[00] : 0.0
m[00] : 2267.0
m[00] : 35.0
m[00] : 107.0
m[00] : 34377.0
processing : 385, Image Name : image_166_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 61474.5
m[00] : 148.5
processing : 386, Image Name : image_282_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 2325.0
m[00] : 101498.5
processing : 387, Image Name : image_589_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 34129.0
processing : 388, Image Name : image_247_pred_binary.png


/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_247_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_406_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_525_cc_binary.png is a low contrast image
Lossy con

[ 30 215]
(512, 512)
[  0 255]
m[00] : 10110.0
m[00] : 37.5
m[00] : 1.5
m[00] : 0.0
m[00] : 0.0
m[00] : 874.0
m[00] : 0.0
m[00] : 11.5
m[00] : 0.0
m[00] : 0.0
m[00] : 0.0
m[00] : 0.0
m[00] : 2013.5
m[00] : 41173.0
processing : 389, Image Name : image_406_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 2.5
m[00] : 65587.5
processing : 390, Image Name : image_525_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 1.0
m[00] : 187.0
m[00] : 0.0
m[00] : 83.5
m[00] : 66052.5
m[00] : 44439.5
processing : 391, Image Name : image_270_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 203174.5
processing : 392, Image Name : image_219_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 136.5
m[00] : 37872.0
processing : 393, Image Name : image_528_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 50.5
m[00] : 47820.5
m[00] : 0.0
m[00] : 251.0
m[00] : 82011.0
m[00] : 61534.0
processing : 394, Image Name : image_482_pred_binary.png
[ 30 215]
(512, 512)
[  0 255]
m[00] : 12291

/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_167_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:23: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-34-cc/image_218_cc_binary.png is a low contrast image
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


### Backup code  - no need to run

In [ ]:
#from utilities.model_errors import ValidationError
def find_centermost_mask(input_mask, min_pixels):
    """
    Take in a binary mask and return a mask of the same shape, with only the centermost contiguous portion.
    If a mask contains no contours (objects), then just return it (it's all zeros anyway).
    """

    # Get all the bits aligned in memory
    input_mask = np.ascontiguousarray(input_mask, dtype=np.uint8)
    contours, _ = cv2.findContours(input_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours or len(contours) == 1:
        return input_mask

    moments = [cv2.moments(contour) for contour in contours]
    # (x, y) tuples of centroid locations for contours (that have area > 0)
    centroids = [
        (m['m10'] / m['m00'], m['m01'] / m['m00'])
        if m['m00'] and m['m00'] >= min_pixels else (0, 0)
        for m in moments
    ]

    if not centroids:
        raise ValidationError(f'No identified footprint contours contain more than {min_pixels} pixels.')

    # dimensions come 1-indexed, but array indicies are 0-indexed
    center = [(dim - 1) / 2 for dim in input_mask.shape]

    # Cacluate (Manhattan) distance from center of image
    distances_from_center = [abs(x - center[0]) + abs(y - center[1]) for x, y in centroids]
    centermost_contour = contours[distances_from_center.index(min(distances_from_center))]

    output_mask = np.zeros(input_mask.shape)
    cv2.fillPoly(output_mask, centermost_contour.transpose((1, 0, 2)), 1)

    return output_mask

In [ ]:
for idx, pred_img_name in enumerate(lst_pred_imgs, start=1):
    print('processing : {}, Image Name : {}'.format(idx, pred_img_name))
    pred_img_path_with_filename = os.path.join(pred_img_path, pred_img_name)
    pred_gray = cv2.imread(pred_img_path_with_filename)
    pred_gray = cv2.cvtColor(pred_gray, cv2.COLOR_BGR2GRAY)
    
#     print(np.unique(pred_gray,  return_counts=True))
#     print(pred_gray.shape)
#     plt.imshow(pred_gray)
#     plt.show()

    pred_gray[pred_gray < 31] = 0
    
#     print(np.unique(pred_gray, return_counts=True))
#     plt.imshow(pred_gray)
#     plt.show()
    
    pred_cc_img_mask=find_centermost_mask(pred_gray,2000)
    pred_cc_img_mask = pred_cc_img_mask.astype(np.byte)
    io.imsave(os.path.join(pred_cc_save_path, lbl_img_name.replace('_pred_binary', '_cc')), pred_cc_img_mask)
#     np.unique(pred_cc_img_mask, return_counts=True)
#     plt.imshow(pred_cc_img_mask)
#     plt.show()